In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np

In [2]:
market = Market()
competition_db = ADatabase("competition")

In [3]:
competition_db.connect()
data = competition_db.retrieve("data")
competition_db.disconnect()

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
corrs = market.retrieve("price_corr")
market.disconnect()

In [6]:
start_year = 2016
end_year = 2023

In [7]:
competition_db.connect()
for training_year in [4]:
    training_corrs = corrs[corrs["training_year"]==training_year]
    for year in tqdm(range(start_year,end_year)):
        for ticker in tqdm(sp5["Symbol"]):
            try:
                if ticker in data.columns:
                    ticker_data = data.copy()
                    example = corrs[(corrs["ticker"]==ticker) & (corrs["year"]==2022) & (corrs["training_year"]==4)].tail(1).fillna(0)[[x for x in corrs.columns if x not in ["ticker","year","training_year"]]].melt()
                    example["abs_corr"] = [abs(x) for x in example["value"]]
                    factors = list(example.sort_values("abs_corr",ascending=False).head(25)["variable"])
                    factors = [x for x in factors if x in ticker_data.columns]
                    training_data = ticker_data[(ticker_data["year"]<year) & (ticker_data["year"]>=year-training_year)]
                    training_data["y"] = training_data[ticker].shift(-1)
                    training_data.dropna(inplace=True)
                    training_data.reset_index(drop=True,inplace=True)
                    prediction_data = ticker_data[ticker_data["year"]==year]
                    if training_data.index.size > 100:
                        refined_data = {"X":training_data[factors],"y":training_data[["y"]]}
                        models = m.regression(refined_data)
                        for row in models.iterrows():
                            model = row[1]["model"]
                            api = row[1]["api"]
                            score = row[1]["score"]
                            prediction_data[f"{api}_prediction"] = model.predict(prediction_data[factors])
                            prediction_data[f"{api}_score"] = score
                        prediction_data["ticker"] = ticker
                        included_columns = ["year","week","ticker","training_year"]
                        included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
                        prediction_data["training_year"] = training_year
                        competition_db.store("sim",prediction_data[included_columns])
                    else:
                        continue
                else:
                    continue
            except Exception as e:
                print(year,ticker,str(e))
competition_db.disconnect()

 11%|█████████████▏                                                                                                             | 54/504 [04:41<35:22,  4.72s/it]


 22%|███████████████████████████▎                                                                                              | 113/504 [09:44<40:46,  6.26s/it]


 33%|████████████████████████████████████████▋                                                                                 | 168/504 [14:45<30:49,  5.50s/it]


 45%|██████████████████████████████████████████████████████▍                                                                   | 225/504 [19:47<28:26,  6.12s/it]


 56%|████████████████████████████████████████████████████████████████████▋                                                     | 284/504 [24:48<18:55,  5.16s/it]


...

 59%|███████████████████████████████████████████████████████████████████████▉                                                  | 297/504 [24:06<14:39,  4.25s/it]


 70%|█████████████████████████████████████████████████████████████████████████████████████▉                                    | 355/504 [28:51<13:45,  5.54s/it]


 82%|███████████████████████████████████████████████████████████████████████████████████████████████████▍                      | 411/504 [33:29<08:46,  5.66s/it]


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌         | 465/504 [38:07<03:16,  5.04s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [4:50:47<00:00, 2492.44s/it]
